In [6]:
import numpy as np
import scipy.sparse as sparse
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Parameters
L = 10  # Domain limit [-L, L] for x and y
n = 64  # New grid points in each direction (updated from previous n=8)
h = (2 * L) / n  # Grid spacing

# Define finite difference stencils for second and first derivatives
second_derivative = np.array([1, -2, 1]) / (h**2)
first_derivative = np.array([-1, 0, 1]) / (2 * h)

# Construct 1D second-derivative matrix with periodic BCs
D2_1d = sparse.spdiags([second_derivative[0] * np.ones(n), 
                        second_derivative[1] * np.ones(n), 
                        second_derivative[2] * np.ones(n)], 
                       [-1, 0, 1], n, n, format="csr")
D2_1d[0, -1] = second_derivative[0]  # Periodic boundary
D2_1d[-1, 0] = second_derivative[2]  # Periodic boundary

# Construct 1D first-derivative matrix with periodic BCs for x
D1x_1d = sparse.spdiags([first_derivative[0] * np.ones(n), 
                         first_derivative[1] * np.ones(n), 
                         first_derivative[2] * np.ones(n)], 
                        [-1, 0, 1], n, n, format="csr")
D1x_1d[0, -1] = first_derivative[0]  # Periodic boundary
D1x_1d[-1, 0] = first_derivative[2]  # Periodic boundary

# Construct 2D operators using Kronecker products
I = sparse.eye(n)  # Identity matrix of size n
A = sparse.kron(I, D2_1d) + sparse.kron(D2_1d, I)  # Laplacian ∂²_x + ∂²_y
C = sparse.kron(I, D1x_1d)                  # First derivative ∂y in 2D
B = sparse.kron(D1x_1d, I)                  # First derivative ∂x in 2D

A = A.toarray()
B = B.toarray()
C = C.toarray()

In [7]:
A

array([[-40.96,  10.24,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [ 10.24, -40.96,  10.24, ...,   0.  ,   0.  ,   0.  ],
       [  0.  ,  10.24, -40.96, ...,   0.  ,   0.  ,   0.  ],
       ...,
       [  0.  ,   0.  ,   0.  , ..., -40.96,  10.24,   0.  ],
       [  0.  ,   0.  ,   0.  , ...,  10.24, -40.96,  10.24],
       [  0.  ,   0.  ,   0.  , ...,   0.  ,  10.24, -40.96]])

In [8]:
import numpy as np
import scipy.sparse as sparse
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Create grid
x2 = np.linspace(-L, L, n + 1)
x = x2[:n]
y2 = np.linspace(-L, L, n + 1)
y = y2[:n]
X, Y = np.meshgrid(x, y)

# Initial vorticity condition (Gaussian)
def initial_vorticity(X, Y):
    return np.exp(-(X**2 + Y**2/20))

# Initial vorticity field
omega0 = initial_vorticity(X, Y).flatten()

# Diffusion coefficient
nu = 0.001

# Compute streamfunction using FFT method
def compute_streamfunction_fft(omega):
    # Implement FFT-based Poisson solver for streamfunction
    omega_2d = omega.reshape((n, n))
    
     # FFT of vorticity
    omega_fft = np.fft.fft2(omega_2d)

    # Create frequency grid
    kx = np.fft.fftfreq(n, h) * 2 * np.pi
    ky = np.fft.fftfreq(n, h) * 2 * np.pi
    KX, KY = np.meshgrid(kx, ky)
    
    # Avoid division by zero
    KX[0, 0] = 1e-6
    KY[0, 0] = 1e-6
    
    # Solve Poisson equation in Fourier space
    psi_fft = -omega_fft / ((KX**2 + KY**2))
    
    # Inverse FFT to get streamfunction
    psi = np.fft.ifft2(psi_fft).real
    
    return psi.flatten()

# Nonlinear Jacobian term [ψ, ω]
def poisson_bracket(psi, omega):
    psi_2d = psi.reshape((n, n))
    omega_2d = omega.reshape((n, n))
    
    # Compute derivatives
    psi_x = (B @ psi_2d.flatten()).reshape((n, n))
    psi_y = (C @ psi_2d.flatten()).reshape((n, n))
    omega_x = (B @ omega_2d.flatten()).reshape((n, n))
    omega_y = (C @ omega_2d.flatten()).reshape((n, n))
    
    return psi_x * omega_y - psi_y * omega_x

# Vorticity evolution equation
def vorticity_rhs_fft(t, omega):
    # Compute streamfunction
    psi = compute_streamfunction_fft(omega)
    
    # Compute Poisson bracket
    bracket = poisson_bracket(psi, omega)
    
    # Diffusion term
    diffusion = (nu * (A @ omega)).reshape(n, n)
    
    return (-bracket + diffusion).flatten()

# Time span
tspan = np.arange(0, 4.5, 0.5)

# Solve using solve_ivp
sol = solve_ivp(vorticity_rhs_fft, [tspan[0], tspan[-1]], omega0, 
                t_eval=tspan, method='RK45')

# Store solution
A1 = sol.y  # Solution for different time steps

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def create_vorticity_movie(X, Y, solution, output_filename):
    # Create figure and axis for animation
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Set fixed color scale based on entire dataset
    vmin = solution.min()
    vmax = solution.max()
    
    # Initial plot setup
    initial_plot = solution[:, 0].reshape((n, n))
    
    # Create initial contour plot with fixed color scale
    contour = ax.contourf(X, Y, initial_plot, levels=20, cmap='viridis', 
                           vmin=vmin, vmax=vmax)
    
    # Add colorbar once
    cbar = plt.colorbar(contour, ax=ax, label='Vorticity')
    
    # Set fixed axis limits
    ax.set_xlim(X.min(), X.max())
    ax.set_ylim(Y.min(), Y.max())
    
    # Animation update function
    def update(frame):
        # Clear only the contour, keep other elements
        for collection in ax.collections:
            collection.remove()
        
        # Plot new vorticity field
        current_plot = A1[:, frame].reshape((n, n))
        contour = ax.contourf(X, Y, current_plot, levels=20, cmap='viridis', 
                               vmin=vmin, vmax=vmax)
        
        # Update title
        ax.set_title(f'Vorticity at t = {tspan[frame]:.2f}')
        
        return contour

    # Create animation
    anim = animation.FuncAnimation(fig, update, frames=len(tspan), 
                                   interval=500, repeat_delay=1000)
    
    # Adjust layout to prevent cutting off elements
    plt.tight_layout()
    
    # Save animation as GIF
    anim.save(output_filename, writer='pillow', fps=2, dpi=100)
    plt.close(fig)
    print(f"Movie saved as {output_filename}")



In [10]:
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import bicgstab, gmres, splu, spsolve
from scipy.linalg import lu, solve_triangular
from scipy.integrate import solve_ivp
import time

A2 = np.zeros(A1.shape)
A3 = np.zeros(A1.shape)
###################
### FFT-Solve #####
###################
print(f"Solving with FFT:")
start_time = time.time() # Record the start time
sol = solve_ivp(vorticity_rhs_fft, [tspan[0], tspan[-1]], omega0, 
                t_eval=tspan, method='RK45')
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

A[0, 0] = 2

######################
#### Direct-Solve ####
######################
# Solve using A\b (direct matrix division)
def compute_streamfunction_direct(A, omega):
    return spsolve(A, omega).flatten()

# Vorticity evolution equation
def vorticity_rhs_direct(t, omega):
    # Compute streamfunction
    psi = compute_streamfunction_direct(A,omega)
    
    # Compute Poisson bracket
    bracket = poisson_bracket(psi, omega)
    
    # Diffusion term
    diffusion = (nu * (A @ omega)).reshape(n, n)
    
    return (-bracket + diffusion).flatten()

######################
###### Lu-Solve ######
######################
# Solve using LU decomposition
def compute_streamfunction_lu(A, omega):
    lu = splu(A)
    x = lu.solve(omega)
    return x.flatten()

# Vorticity evolution equation
def vorticity_rhs_lu(t, omega):
    # Compute streamfunction
    psi = compute_streamfunction_lu(A, omega)
    
    # Compute Poisson bracket
    bracket = poisson_bracket(psi, omega)
    
    # Diffusion term
    diffusion = (nu * (A @ omega)).reshape(n, n)
    
    return (-bracket + diffusion).flatten()

############################
###### BICGSTAB-Solve ######
############################
# Solve using BICGSTAB
def compute_streamfunction_bicgstab(A, omega, tol=1e-6, max_iter=500):
    x, exitCode = bicgstab(A, omega, tol=tol, maxiter=max_iter, callback=callback_bicgstab)
    return x.flatten()

# Vorticity evolution equation
def vorticity_rhs_bicgstab(t, omega):
    # Compute streamfunction
    psi = compute_streamfunction_bicgstab(A, omega)
    
    # Compute Poisson bracket
    bracket = poisson_bracket(psi, omega)
    
    # Diffusion term
    diffusion = (nu * (A @ omega)).reshape(n, n)
    
    return (-bracket + diffusion).flatten()

#########################
###### GMRES-Solve ######
#########################
# Solve using GMRES
def compute_streamfunction_gmres(A, omega, tol=1e-6, max_iter=500):
    x, exitCode = gmres(A, omega, tol=tol, maxiter=max_iter, callback=callback_gmres)
    return x.flatten()

# Vorticity evolution equation
def vorticity_rhs_gmres(t, omega):
    # Compute streamfunction
    psi = compute_streamfunction_gmres(A, omega)
    
    # Compute Poisson bracket
    bracket = poisson_bracket(psi, omega)
    
    # Diffusion term
    diffusion = (nu * (A @ omega)).reshape(n, n)
    
    return (-bracket + diffusion).flatten()

residuals_gmres = []
residuals_bicgstab = []

def callback_gmres(rk):
    """Callback to record residual norm per iteration for GMRES."""
    residuals_gmres.append(rk)

def callback_bicgstab(rk):
    """Callback for BICGSTAB - not directly residual norm per iteration."""
    residuals_bicgstab.append(np.linalg.norm(rk))  # Convert residual to norm

# Record time for different methods
def time_method(method, A, omega, *args, **kwargs):
    start_time = time.time()
    if method == 'direct':
        solution = solve_ivp(vorticity_rhs_direct, [tspan[0], tspan[-1]], omega0, 
                t_eval=tspan, method='RK45')
    elif method == 'LU':
        solution = solve_ivp(vorticity_rhs_lu, [tspan[0], tspan[-1]], omega0, 
                t_eval=tspan, method='RK45')
    # elif method == 'BICGSTAB':
    #     solution = solve_ivp(vorticity_rhs_bicgstab, [tspan[0], tspan[-1]], omega0, 
    #             t_eval=tspan, method='RK45')
    # elif method == 'GMRES':
    #     solution = solve_ivp(vorticity_rhs_gmres, [tspan[0], tspan[-1]], omega0, 
    #             t_eval=tspan, method='RK45')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"{method} Elapsed time: {elapsed_time:.2f} seconds")
    
    return solution, elapsed_time

# Run the solver and compare the results
methods = ['direct', 'LU'] # , 'BICGSTAB', 'GMRES']
method_times = {}
for method in methods:
    print(f"Solving with {method}:")
    solution, elapsed_time = time_method(method, A, omega0)
    if(method == 'direct'):
        A2 = solution.y
    if(method == 'LU'):
        A3 = solution.y

# Plot residuals vs iterations
# plt.figure(figsize=(8, 6))
# for i in range(1, len(residuals_gmres)):
#     plt.plot(i, residuals_gmres[i], marker='o', label="GMRES")
# for i in range(1, len(residuals_gmres)):
#     plt.plot(i, residuals_bicgstab[i], marker='x', label="BICGSTAB")
# plt.yscale('log')
# plt.xlabel('Iterations')
# plt.ylabel('Residual Norm (log scale)')
# plt.title('Residual Norm vs Iterations')
# plt.grid()
# plt.legend()
# plt.show()


Solving with FFT:
Elapsed time: 0.37 seconds
Solving with direct:


/var/folders/gl/2q9xsmfd5ll6rjdlnqfslpqr0000gn/T/ipykernel_65572/1817103980.py:28: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  return spsolve(A, omega).flatten()


direct Elapsed time: 2.61 seconds
Solving with LU:


/var/folders/gl/2q9xsmfd5ll6rjdlnqfslpqr0000gn/T/ipykernel_65572/1817103980.py:48: SparseEfficiencyWarning: splu converted its input to CSC format
  lu = splu(A)


LU Elapsed time: 2.54 seconds


In [11]:
A2

array([[ 2.50656748e-46, -1.78784704e-36,  1.02706391e-36, ...,
        -5.76806489e-26, -6.68287054e-25,  3.02050167e-24],
       [ 1.17762859e-43, -2.26093387e-29,  1.85614646e-29, ...,
        -1.48306461e-19, -7.82424220e-19,  1.67449651e-18],
       [ 4.55107657e-41, -4.24661881e-29,  3.48720993e-29, ...,
        -2.20165937e-20, -2.91637720e-19,  1.23961335e-18],
       ...,
       [ 1.96785570e-38,  7.20814729e-28, -6.58759786e-28, ...,
         4.01674620e-20,  1.34371413e-19, -8.94163775e-19],
       [ 6.19028421e-41,  1.69117621e-28, -1.49205998e-28, ...,
         3.00460834e-20,  1.76009486e-19, -9.80736832e-19],
       [ 1.60178709e-43,  5.05834665e-29, -4.38046443e-29, ...,
         2.47633369e-20,  1.15181522e-20,  3.12761254e-20]])

In [12]:
A3

array([[ 2.50656748e-46, -1.78784704e-36,  1.02706391e-36, ...,
        -5.76806489e-26, -6.68287054e-25,  3.02050167e-24],
       [ 1.17762859e-43, -2.26093387e-29,  1.85614646e-29, ...,
        -1.48306461e-19, -7.82424220e-19,  1.67449651e-18],
       [ 4.55107657e-41, -4.24661881e-29,  3.48720993e-29, ...,
        -2.20165937e-20, -2.91637720e-19,  1.23961335e-18],
       ...,
       [ 1.96785570e-38,  7.20814729e-28, -6.58759786e-28, ...,
         4.01674620e-20,  1.34371413e-19, -8.94163775e-19],
       [ 6.19028421e-41,  1.69117621e-28, -1.49205998e-28, ...,
         3.00460834e-20,  1.76009486e-19, -9.80736832e-19],
       [ 1.60178709e-43,  5.05834665e-29, -4.38046443e-29, ...,
         2.47633369e-20,  1.15181522e-20,  3.12761254e-20]])

In [13]:
# # Create vorticity movie
# create_vorticity_movie(X, Y, A1, 'vorticity_evolution_fft.gif')

In [14]:
# import numpy as np
# import scipy.sparse as sparse
# from scipy.integrate import solve_ivp
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation

# # Random assortment function
# def random_vortices(X, Y, num_vortices=10):
#     omega = np.zeros_like(X)
#     for _ in range(num_vortices):
#         x0 = np.random.uniform(-L, L)
#         y0 = np.random.uniform(-L, L)
#         strength = np.random.uniform(-1, 1)
#         ellipticity = np.random.uniform(1, 3)
#         omega += strength * np.exp(-((X - x0)**2 + (Y - y0)**2 / ellipticity))
#     return omega

# # Define initial vorticity for different cases
# def initial_vorticity(X, Y, case):
#     if case == 1:  # Oppositely charged vortices
#         return (np.exp(-((X - 2)**2 + (Y**2)/20)) - np.exp(-((X + 2)**2 + (Y**2) / 20))).flatten()
#     elif case == 2:  # Same charged vortices
#         return (np.exp(-((X - 2)**2 + (Y**2) / 20)) + np.exp(-((X + 2)**2 + (Y**2) / 20))).flatten()
#     elif case == 3:  # Pairs of oppositely charged vortices
#         return (
#             np.exp(-((X - 2)**2 + ((Y - 2)**2))) - np.exp(-((X + 2)**2 + ((Y - 2)**2)))
#             + np.exp(-((X - 2)**2 + ((Y + 2)**2))) - np.exp(-((X + 2)**2 + ((Y + 2)**2)))
#         ).flatten()
#     elif case == 4:  # Random assortment
#         return random_vortices(X, Y, num_vortices=15).flatten()


# # Solve for each case
# cases = [1, 2, 3, 4]
# solutions = []

# for case in cases:
#     omega0 = initial_vorticity(X, Y, case)
#     sol = solve_ivp(vorticity_rhs_fft, [tspan[0], tspan[-1]], omega0, t_eval=tspan, method='RK45')
#     solutions.append(sol.y)

# # Animation creation
# def create_animation(solution, filename):
#     fig, ax = plt.subplots()
#     vorticity_plot = ax.imshow(solution[:, 0].reshape(n, n), extent=(-L, L, -L, L), cmap='viridis', animated=True)
#     plt.colorbar(vorticity_plot, ax=ax)

#     def update(frame):
#         vorticity_plot.set_array(solution[:, frame].reshape(n, n))
#         return [vorticity_plot]

#     ani = animation.FuncAnimation(fig, update, frames=solution.shape[1], interval=200, blit=True)
#     ani.save(filename, writer='pillow', fps=10, dpi=300)
#     plt.close(fig)

# # Generate and save animations
# filenames = [
#     "opposite_vortices.gif",
#     "same_vortices.gif",
#     "colliding_vortices.gif",
#     "random_vortices.gif",
# ]

# for i, solution in enumerate(solutions):
#     create_animation(solution, filenames[i])

# print("All animations saved successfully.")
